In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [34]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

def mean(numbers):
    return int(round (sum(numbers)) / max(len(numbers), 1))

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[100, 255, 0], thickness=12):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    color2=[0, 0, 255]
    thickness2=12

    x1right = []
    x2right = []
    y1right = []
    y2right = []
    x1left  = []
    x2left  = []
    y1left  = []
    y2left  = []     

    for line in lines:
        for x1,y1,x2,y2 in line:
            #obtain the lines slope sign
            slope=(y2-y1)/(x2-x1)
            if slope>0:
              x1right.append(x1)
              y1right.append(y1)
              x2right.append(x2)
              y2right.append(y2)     
            else:#slope<0  
              x1left.append(x1)
              y1left.append(y1)
              x2left.append(x2)
              y2left.append(y2)

    if   not x1right:
     print("right lines list is empty")
    else:
     cv2.line(img, (mean(x1right), mean(y1right)), (mean(x2right), mean(y2right)),color2, thickness2)
    if   not x1left:
     print("left lines list is empty")
    else:
     cv2.line(img, (mean(x1left ), mean( y1left)), (mean(x2left ), mean( y2left)), color,  thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [51]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    # TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
#image path and valid extensions
#import os
#imagespath = "test_images/" 
#imageslist = []
#valid_image_extensions = [".jpg", ".jpeg", ".png", ".tif", ".tiff"] 
#valid_image_extensions = [item.lower() for item in valid_image_extensions]
#create a list all files in directory and
#append files with a vaild extention to image_path_list
#for file in os.listdir(imagespath):
#    extension = os.path.splitext(file)[1]
#    if extension.lower() not in valid_image_extensions:
#        continue
#   imageslist.append(os.path.join(imagespath, file))
#loop
#for imagePath in imageslist:
#    image = cv2.imread(imagePath)
    
    # show
    #if image is not None:
        #cv2.imshow(imagePath, image)
    #elif image is None:
       # print ("read error: " + imagePath)
       #continue
    
    #/////////////////image process///////////////
    # color transform
    grayimg   = grayscale(image)
    #cv2.imshow("gray", grayimg)
    # denoise
    denoiseimg=gaussian_blur(grayimg, 5)
    #cv2.imshow("denoise", denoiseimg)
    # border extraction
    low_threshold=50
    high_threshold=150
    bordersimage=canny( denoiseimg, low_threshold, high_threshold)
    #cv2.imshow("canny", bordersimage)
    #mask
    imshape = image.shape #return height, width, channels
    vertices = np.array([[(0,imshape[0]),((imshape[1]/2)-20, (imshape[0]/2)+40), ((imshape[1]/2)+20, (imshape[0]/2)+40), (imshape[1],imshape[0])]], dtype=np.int32)
    maskimg=region_of_interest(bordersimage, vertices)
    #cv2.imshow("maskimg", maskimg)
    #find lines 
    rho=1
    theta=(np.pi/180)*0.4
    threshold=30
    min_line_len=100
    max_line_gap=150
    linesimage=hough_lines(maskimg, rho, theta, threshold, min_line_len, max_line_gap)
    #cv2.imshow("hough lines", linesimage)
    #visualize
    α=0.6
    β=1-α
    λ=0.0
    weightedimage=weighted_img(image, linesimage,α,β,λ)
    #cv2.imshow("weighted lines", weightedimage)

    #/////////////////////////////////////////////

    #key = cv2.waitKey(0)
    #if key == 27: # escape
        #break
    #cv2.destroyAllWindows()
# close  windows
#cv2.destroyAllWindows()
    
    
    
    return weightedimage

In [52]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 221/222 [00:05<00:00, 39.21it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 6.39 s


In [53]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [54]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:21<00:00, 31.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 22.5 s


In [55]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [ ]:
Reflection on the project:
    
In the pipeline computer vision algorithms has been applied
in six stages , in the first a color space transformation to
gray scale , this reduce RGB image to one channel image
through a weighted sum, in the second stage the gray
image was smoothed with gaussian filter for noise reduction,
the third stage uses a canny edge detector with thresold
whose range was established from static images, then a
polygonal mask was configured to preserve only the information
on the road. In the five stage a hough transform is used to
detect straight lines and finally the result is Interlaced
with original image for visualization purposes.

This initial solution is only applicable for images with
similar characteristics of lighting, perspective of the
camera and content since for other conditions are recalculated
the parameters for the stages, it is because this does not work
properly for the video extra.mp4

It is possible to include other image preprocessing steps to try
to normalize the input images such as histogram equalization
or generate a table of variable parameters for the thresholds of
canny detector according to the minimum and maximum levels of
the image histogram with adaptive thresholding, however, these
techniques are not very robust to unstructured environments,
which requires algorithms with a certain degree of "intelligence"
    

In [56]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:15<00:00, 23.21it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 16.2 s


In [57]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))